In [7]:
!pip install langchain
!pip install pypdf
!pip install unstructured
!pip install sentence_transformers
!pip install pinecone-client
!pip install pdf2image
!pip install pdfminer
!pip install pdfminer.six
!pip install openai
!pip install tiktoken


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip insta

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI

import pinecone
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-east1-gcp')
os.environ['OPENAI_API_KEY'] = ""

In [9]:
loader = PyPDFLoader("https://www.ema.europa.eu/en/documents/regulatory-procedural-guideline/user-guide-how-generate-pdf-versions-product-information-veterinary_en.pdf")

In [10]:
data = loader.load()

In [11]:
data

[Document(page_content=' \n.  \nOfficial address   Domenico Scarlattilaan 6   ●  1083 HS Amsterdam   ●  The Netherlands   \n An agency of the European Union      \n  Address for visits and deliveries   Refer to www.ema.europa.eu/how -to-find-us  \nSend us a question  Go to www.ema.europa.eu/contact   Telephone +31 (0)88 781  6000 \n \n \n© European Medicines Agency,  2023. Reproduction is authorised provided the source is acknowledged.  12 May 2023  \nEMA/793983/2010 v. 4.01 \nVeterinary Medicines  Division  \nUser guide on how to generate PDF versions of the \nproduct information - veterinary \nIntroduction  \nSince the product information consists of the combination of summary of product characteristics \n(Annex I), labelling (Annex IIIA), package leaflet (Annex IIIB) and conditions of the marketing authorisation (Annex II), the PDF version of the product information requires bookmarks to identify the \nlocation of these different Annexes with in the single file. File properties are 

In [12]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

In [42]:
docs=text_splitter.split_documents(data)

In [43]:
len(docs)

31

In [76]:
import os
# PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-east1-gcp')


from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [47]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)
index_name = "index-1"

In [48]:
#New
# docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

#Current
# docsearch = Pinecone.from_existing_index(index_name, embeddings)
docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [49]:
query="Whas is the advantage of Method 2?"

In [50]:
docs=docsearch.similarity_search(query)

In [51]:
docs

[Document(page_content='specific line in the text. It is also possible to select more than one full product  name  in one bookmark.  \nMethod 2 has the advantage that properties and bookmarks need only be defined once in the Word file \nfor each language version; they will be maintained for each subsequent revision of the product \ninformation and are automatically carried over in  the converted PDF files.  \nThe two methods are explained below.  \nMethod 1: setting properties and bookmarks in the generated PDF files', metadata={'page': 0.0, 'source': '/var/folders/wb/z0x047fj5llby_8fmwk6v9xm0000gn/T/tmpyfwh6p_m/tmp.pdf'}),
 Document(page_content='Method 2: setting properties and bookmarks in the Word files and conversion to PDF  \n1. Preparation of the Word source files', metadata={'page': 3.0, 'source': '/var/folders/wb/z0x047fj5llby_8fmwk6v9xm0000gn/T/tmpyfwh6p_m/tmp.pdf'}),
 Document(page_content='2 In case of product information with more t han one package leaflet: steps 5 and 6 n

In [62]:
llm=ChatOpenAI(model="gpt-4")

In [63]:
chain=load_qa_chain(llm, chain_type="stuff")

In [73]:
query="Whas is the advantage of Method 2"
docs=docsearch.similarity_search(query)

In [72]:
chain.run(input_documents=docs, question=query)

"I'm sorry, but the provided context doesn't provide information on examples of good data science teams."

In [69]:
query="What are examples of good data science teams?"
docs=docsearch.similarity_search(query)
docs

[Document(page_content='13. Click on the Description  tab and enter the following information, for example for a \nproduct with the invented name ‘Imaginary’ and with INN ‘examplamin  sodium’  (for \nvaccine products only the product name should be mentioned) :  \n• Title: Imaginary, INN -examplamin sodium  \n• Subject: EPAR  \n• Author: CVMP  \n• Keywords: Imaginary, INN -examplamin sodium', metadata={'page': 2.0, 'source': '/var/folders/wb/z0x047fj5llby_8fmwk6v9xm0000gn/T/tmpyfwh6p_m/tmp.pdf'}),
 Document(page_content='7. In case of product information with more than one SPC  (i.e several products compiled \nin one set of annexes) : steps 5 and 6 also need to be repeated for the text under section \n1 in each SPC: \ne.g. \nImaginary 10 mg/ml solution for injection for cattle and pigs  \nImaginary 2 mg/ml oral suspension for dogs  \nIn case more than one full product  name  is listed under section 1 all of them s hould be \nbookmarked : \ne.g. \nImaginary 1 mg chewable tablets for dog

In [70]:
chain.run(input_documents=docs, question=query)

"I'm sorry, but the information provided does not include any details about good data science teams."